# Generador de Tablas de Verdad en LaTex / LaTex Truth Tables Generator

Se utliza un árbol para representar la la estructura de una proposición lógica. Los métodos incluidos toman cadenas de caracteres como entrada. Están considerados los siguientes operadores:

- Negación `¬` 
- Conjunción `^`
- Disyunción `|`
- Condicional `->`
- Bicondicional `<->`

Se deben usar paréntesis para evitar ambigüedades. Por ejemplo, $a\wedge b\implies c$ debe escribirse como `"(a^b)->c"`. También, las variables deben ser letras del alfabeto.

Las funciones disponibles para generar tablas de verdad, son`texTT` y `mainTexTT`. 

`texTT` toma los siguientes argumentos:
```
    texTT(filename, str_expr, standalone)
```
donde `filename` es el nombre del archivo a crear, `str_expr` es una cadena que contiene la proposición a calcular y `standalone` es una variable de tipo `bool` que especifica si se van a incluir o no los paquetes y el ambiente `document` en el archivo.

`mainTexTT` se usa para agregar varias tablas de verdad en el mismo documento base de LaTex y toma los siguientes argumentos:
```
    texTT(filename, exprs)
```
donde `filename` es el nombre del archivo a crear y `exprs` es una lista que contiene varias cadenas de caracteres con las proposiciones a calcular.


---


A tree is used to represent the structure of a logical proposition. The buit methods take strings as input. The following operators are supported:

- Negation `¬` 
- Conjunction `^`
- Disjunction `|`
- Conditional `->`
- Biconditional `<->`

Parentheses must be used to avoid ambiguity. For example, $a\wedge b\implies c$ must be written as `"(a^b)->c"`. Also, all variables must be letters of the alphabet.

The available functions to create truth tables are `texTT` and `mainTexTT`. 

`texTT` is used to create a single truth table and takes the following arguments:
```
    texTT(filename, str_expr, standalone)
```
where `filename` is the name of the file to create, `str_expr` is a string with the logical expression to calculate and `standalone` is a variable of type `bool` that specifies whether or not to include packages and the `document` environment in the file.

`mainTexTT` is used to add several TruthTables to the same LaTex main document and takes the following arguments:
```
    texTT(filename, exprs)
```
where `filename` is the name of the file to create and `exprs` is a list that contains several strings with the logical expressions to calculate.



## Clases / Classes

In [1]:
import string

In [2]:
class Node:
    def __init__(self, data, *args):
        self.data = data
        self.children = []
        self.name = ""
        self.classname = "Node"
        self.complete = True
        self.noperator = 0
        
        if len(args)>0:
            self.name=args[0]
    
    def __str__(self):
        s = self.classname[:1] + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    def __repr__(self):
        s = self.classname + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    
    def __eq__(self, other):
        return self.name == other.name
    
    def printTreeNodes(self):
        if self.noperator == 0: #variable
            print(self)
        elif self.noperator == 1: #Operador unario
            print(self)
            self.children[0].printTreeNodes()
        elif self.noperator == 2: #Operador binario
            self.children[0].printTreeNodes()
            print(self)
            self.children[1].printTreeNodes()
        else: #Paréntesis
            print(self)
            self.children[0].printTreeNodes()
            print(")")
            
    def __stringExpressionRec__(self, s):
        cadena = s
        if self.noperator == 0: #variable
            cadena += self.name
        elif self.noperator == 1: #Operador unario
            cadena += self.name
            cadena += self.children[0].__stringExpressionRec__(s)
        elif self.noperator == 2: #Operador binario
            cadena += self.children[0].__stringExpressionRec__(s)
            cadena += self.name
            cadena += self.children[1].__stringExpressionRec__(s)
        else: #Paréntesis
            cadena += "("
            cadena += self.children[0].__stringExpressionRec__(s)
            cadena += ")"
        return cadena
    def stringExpression(self):
        return self.__stringExpressionRec__("")
    def printExpression(self):
        print(self.stringExpression())
    
    def setValue(self, value):
        self.data = value

    def isComplete(self):
        return self.complete
    
    def setComplete(self):
        self.complete = True
        
    def insert(self, N):
        self.children.append(N)
    
    def function(self):
        return self.data

In [3]:
class Parentesis(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Parentesis"
        self.complete = False
        self.noperator = -1
    
    def function(self):
        return self.children[0].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 1.")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [4]:
class Conjuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Conjuncion"
        self.name = "^"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return self.children[0].function() and self.children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 2.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [5]:
class Disyuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Disyuncion"
        self.name = "|"
        self.complete = False
        self.noperator = 2
        
    def function(self):
        return self.children[0].function() or self.children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 3.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [6]:
class Negacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Negacion"
        self.name = "¬"
        self.complete = False
        self.noperator = 1
    
    def function(self):
        return not self.children[0].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 4.")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [7]:
class Implicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Implicacion"
        self.name = "->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        if self.children[0].function()==True and self.children[1].function()==False:
            return False
        else:
            return True

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 5.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

In [8]:
class DobleImplicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "DobleImplicacion"
        self.name = "<->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return self.children[0].function()==self.children[1].function()

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 6.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

## Functions

In [9]:
def string2Proposition(str_prop):
    Letras = string.ascii_lowercase + string.ascii_uppercase
    valid_chars = "|^¬ <->()"+Letras
    Variables = []
    node_stack = []
    j = None
    i=0
    while i < len(str_prop):        
        if str_prop[i] == ' ':
            i+=1
            continue
        
        #Agregar el nodo al stack, sea cual sea
        if str_prop[i]=='(':
            node_stack.append(Parentesis(None))
            i+=1
            continue
        
        if str_prop[i] in Letras:
            if str_prop[i] not in map(lambda x: x.name, Variables):
                Variables.append(Node(None, str_prop[i]))
            j = Variables.index(Node(None, str_prop[i]))   
            node_stack.append(Variables[j])
        
        if str_prop[i] == '¬':
            node_stack.append(Negacion(None))
            
        if str_prop[i] == '^':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 1")
                return None
            node_stack.append(Conjuncion(None))
    
        if str_prop[i] == '|':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 2")
                return None
            node_stack.append(Disyuncion(None))
        
        if str_prop[i] == '-':
            if (i >= len(str_prop)-1) or str_prop[i+1]!='>':
                print("String error 3")
                return None
            node_stack.append(Implicacion(None))
            i+=1 #Porque este operador es de dos caracteres
        
        if str_prop[i] == '<':
            if (i >= len(str_prop)-2) or str_prop[(i+1):(i+1+2)]!="->":
                print("String error 4")
                return None
            node_stack.append(DobleImplicacion(None))
            i+=2 #Porque este operador es de tres caracteres
        
        
        #Caso especial: Cierre de paréntesis
        if str_prop[i] == ')':
            #Asegurar que no haya habido algún error con la cadena
            if type(node_stack[-2])!= Parentesis:
                print("String error 5")
                return None
            #Agregar el contenido del paréntesis dentro del nodo paréntesis
            node_stack[-2].insert(node_stack[-1])
            node_stack.remove(node_stack[-1])
            
            #Si es que hay un operador antes del paréntesis, agregar el contenido del paréntesis al operador.
            if (len(node_stack)>1) and (node_stack[-2].noperator >0):
                node_stack[-2].insert(node_stack[-1])
                node_stack.pop()
            i+=1
            continue
        
        #Revisar qué hacer con el elemento que se acaba de añadir al stack
        #Variables:
        if node_stack[-1].noperator == 0:
            #Casos en los que no hay que meter al nodo dentro de otro todavía
            if (len(node_stack)==1) or (type(node_stack[-2])==Parentesis):
                i+=1
                continue
            #Casos en los que hay un operador antes de la variable
            node_stack[-2].insert(node_stack.pop())
            i+=1
            continue
            
        #Operadores unarios:
        if (node_stack[-1].noperator == 1) and (type(node_stack[-1])!=Parentesis):
            i+=1
            continue #No hay que hacer nada ya que no se vincula con lo que hay antes de él
            
        #Operadores binarios:
        if node_stack[-1].noperator == 2:
            node_stack[-1].insert(node_stack[-2])
            del node_stack[-2]
        
        i+=1
        
    
    return (Variables, node_stack[0])

In [10]:
def recursiveTree2Lists(node, funcs, exprs):
    if node.noperator == 2:
        recursiveTree2Lists(node.children[0], funcs, exprs)
        recursiveTree2Lists(node.children[1], funcs, exprs)
        funcs.append(node.function)
        exprs.append(node.stringExpression())
    elif node.noperator == 1:
        funcs.append(node.function)
        exprs.append(node.stringExpression())
        recursiveTree2Lists(node.children[0], funcs, exprs)
    elif type(node)==Parentesis:
        recursiveTree2Lists(node.children[0], funcs, exprs)
    else: #Variables
        pass
    
        

In [11]:
def expressionLists(exp_tree):
    
    func_list = []
    expr_list = []
    recursiveTree2Lists(exp_tree, func_list, expr_list)
    
    return (func_list, expr_list)

In [12]:
def texHeader(file):
    file.write("\\documentclass[12pt]{article}\r\n\\usepackage[utf8]{inputenc}\r\n\\pagestyle{plain}\r\n\\usepackage[cm]{fullpage}\r\n\\usepackage{amsmath}\r\n\\usepackage{float}\r\n\\usepackage{array}\r\n\r\n")

In [13]:
def convertStr2LaTex(str_prop):
    Letras = string.ascii_lowercase + string.ascii_uppercase
    valid_chars = "|^¬ <->()"+Letras
    
    str2 = "$"
    i=0
    while(i<len(str_prop)):
        if str_prop[i] not in valid_chars:
            print("Not suported character:", str_expr[i])
            return None
        
        if str_prop[i]=='(':
            str2 += "("
        elif str_prop[i] in Letras:
            str2 += str_prop[i]
        elif str_prop[i] == '¬':
            str2 += r"\neg "
        elif str_prop[i] == '^':
            str2 += r"\wedge "
        elif str_prop[i] == '|':
            str2 += r"\vee "
        elif str_prop[i] == '-':
            if (i >= len(str_prop)-1) or str_prop[i+1]!='>':
                print("String error 3")
                return None
            str2 += r"\implies "
            i+=1 #Porque este operador es de dos caracteres
        elif str_prop[i] == '<':
            if (i >= len(str_prop)-2) or str_prop[(i+1):(i+1+2)]!="->":
                print("String error 4")
                return None
            str2 += r"\iff "
            i+=2 #Porque este operador es de tres caracteres
        #Caso especial: Cierre de paréntesis
        elif str_prop[i] == ')':
            str2 += ")"
        i+=1
    str2 += "$"
    return str2

In [14]:
def laTexTrueFalse(value):
    return "$V$" if value else "$F$"

In [35]:
#Función para crear un tabular en latex con una tabla de verdad
def texTT(filename, str_expr, standalone):
    #Obtener funciones y nombres de columnas
    (variables, tree) = string2Proposition(str_expr)
    (func_list, expr_list) = expressionLists(tree)
    #Agregar las variables al comienzo de la tabla
    func_list = list(map(lambda x:x.function, variables)) + func_list
    expr_list = list(map(lambda x:x.stringExpression(), variables)) + expr_list
    expr_list = list(map(convertStr2LaTex, expr_list))
    #Crear tabla en archivo
    file = open(filename, "w")
    
    #Latex pre
    if standalone:
        texHeader(file)
        file.write("\\begin{document}\n")
    file.write("\\begin{center}\n")
    columnformat="{|"
    for v in expr_list:
        columnformat+="c|"
    columnformat+="}"
    file.write("\t\\begin{tabular}"+columnformat+"\n")
    
    #Título de la tabla:
    file.write("\t\t\\hline\n\t\t")
    for i in range(len(expr_list)):
        file.write(expr_list[i])
        if i != len(expr_list)-1:
            file.write(" & ")
        
    file.write("\\\\ \n\t\t\\hline\n")
    
    #Ciclo principal de la tabla de verdad
    N = len(variables)
    for i in range(2**N):
        valores = [bool(int(x)) for x in bin(i)[2:].zfill(N)]
        for j in range(N):
            variables[j].setValue(valores[j])
        file.write("\t\t")
        for j in range(len(func_list)):
            file.write(laTexTrueFalse(func_list[j]()))
            if j<len(func_list)-1:
                file.write(" & ")
        file.write("\\\\ \n\t\t\\hline\n")
    #Latex post
    file.write("\\end{tabular}\\end{center}\n")
    if standalone:
        file.write("\\end{document}\n")
    
    file.close()
    print("Se ha creado el archivo '"+filename+"" )
    

In [38]:
#Función para crear un archivo de LaTex con varias tablas de verdad
def mainTexTT(filename, exprs):
    if len(exprs)==0:
        print("Empty expressions lists")
        return None
    
    mainfile = open(filename, "w")
    texHeader(mainfile)
    mainfile.write("\\begin{document}\n")
        
    for i in range(len(exprs)):
        name = filename[:-4]+"_"+str(i+1)+".tex"
        texTT(name, exprs[i], False)
        mainfile.write("\\input{"+name+"}\n")        
        
    mainfile.write("\\end{document}")
    mainfile.close()
    print("Se ha creado el archivo '"+filename+"'")

# Examples

In [40]:
mainTexTT("Prueba.tex", ["¬a", "a^b", "a|b", "a->b", "a<->b", "(a<->b)->((c^a)<->(c^b))"])

Se ha creado el archivo 'Prueba_1.tex
Se ha creado el archivo 'Prueba_2.tex
Se ha creado el archivo 'Prueba_3.tex
Se ha creado el archivo 'Prueba_4.tex
Se ha creado el archivo 'Prueba_5.tex
Se ha creado el archivo 'Prueba_6.tex
Se ha creado el archivo 'Prueba.tex'
